In [45]:
import pandas as pd
import numpy as np
import datetime

In [46]:
prod_plan_lab = 'MB'
prod_start_date = pd.datetime(2019, 10, 21)
prod_end_date = pd.datetime(2019, 11, 10)
prod_date_range = pd.date_range(start=prod_start_date, end=prod_end_date)
prod_date_range

DatetimeIndex(['2019-10-21', '2019-10-22', '2019-10-23', '2019-10-24',
               '2019-10-25', '2019-10-26', '2019-10-27', '2019-10-28',
               '2019-10-29', '2019-10-30', '2019-10-31', '2019-11-01',
               '2019-11-02', '2019-11-03', '2019-11-04', '2019-11-05',
               '2019-11-06', '2019-11-07', '2019-11-08', '2019-11-09',
               '2019-11-10'],
              dtype='datetime64[ns]', freq='D')

In [47]:
def tat_met_calc(row):
    if row['output_date']<=row['tat_target_date']:
        return row['stone_count']
    else:
        return 0

In [48]:
def stone_unit_to_count(row):
    dd = (row['dd_mix']/2)/((row['dd_mix']/2)+(1-row['dd_mix']))
    dg = 1 - dd
    dd_stone = dd * row['plotted_unit'] * 2
    dg_stone = dg * row['plotted_unit']
    stone_count = dd_stone + dg_stone
    return stone_count

In [49]:
df_weekmask = pd.read_excel('SC Projections.xlsx', sheet_name='weekmask')
weekmask = df_weekmask.loc[(df_weekmask.location==prod_plan_lab)]['weekmask'].astype(str)
weekmask = weekmask.values[0]
weekmask

'1111100'

In [50]:
df_holiday = pd.read_excel('SC Projections.xlsx', sheet_name='holiday')
holidays = df_holiday[(df_holiday.location==prod_plan_lab)& (df_holiday.holiday_date.isin(prod_date_range))]['holiday_date'].values.astype('datetime64[D]')
print(holidays)

['2019-10-21' '2019-10-28' '2019-10-29' '2019-10-30' '2019-10-31'
 '2019-11-01']


In [51]:
df_min_output = pd.read_excel('SC Projections.xlsx', sheet_name='min_output')
sc_days = 1
min_output = df_min_output[(df_min_output.current_location==prod_plan_lab) & (df_min_output.so_to_sc_days==sc_days)]['stone_units'].values
print(sc_days, min_output )

if min_output.size>0:
    print('list is not empty')
else:
    print('list is empty')

1 [3813]
list is not empty


In [52]:
df_intake = pd.read_excel('SC Projections.xlsx', sheet_name='intake_data', parse_dates=['tat_start_date', 'service_request_date', 'intake_date', 'tat_target_date', 'erd_target_date'])
df_intake = df_intake[ (df_intake.current_location==prod_plan_lab) & (df_intake.tat_start_date<=prod_end_date) ]
df_intake['plot_rank'] = df_intake[['tat_target_date', 'tat_start_date', 'service_request_date', 'erd_target_date', 'intake_date']].apply(tuple, axis=1).rank(method='dense', ascending=True)
df_intake['unit_balance'] = df_intake['stone_units']
df_intake.sort_values(by=['tat_start_date','plot_rank'])
df_intake.head()

,current_location,intake_location,tat_start_date,service_request_date,intake_date,stone_count,dd_mix,stone_units,tat_target_date,erd_target_date,input_index,plot_rank,unit_balance
0,MB,MB,2019-10-16,2019-10-16,2019-10-16,1558,0.77,958.170,2019-10-18,2019-10-24,0,1.0,958.170
1,MB,MB,2019-10-17,2019-10-17,2019-10-17,7940,0.77,4883.100,2019-10-22,2019-10-25,1,2.0,4883.100
2,MB,MB,2019-10-18,2019-10-18,2019-10-18,8867,0.77,5453.205,2019-10-23,2019-11-04,2,3.0,5453.205
3,MB,MB,2019-10-19,2019-10-19,2019-10-19,4788,0.77,2944.620,2019-10-23,2019-11-04,3,4.0,2944.620
4,MB,MB,2019-10-22,2019-10-22,2019-10-22,13472,0.77,8285.280,2019-10-24,2019-11-05,4,5.0,8285.280


In [53]:
df_output = pd.read_excel('SC Projections.xlsx', sheet_name='output_data', parse_dates=['production_date'])
df_output = df_output[ (df_output.current_location==prod_plan_lab) & (df_output.production_date.isin(prod_date_range)) ]
df_output['unit_balance'] = df_output['stone_units']
df_output.sort_values(by='production_date')
df_output.head()

,current_location,production_date,output_capacity,dd_mix,stone_units,output_index,unit_balance
0,MB,2019-10-22,10500,0.77,6457.500,0,6457.500
1,MB,2019-10-23,10500,0.77,6457.500,1,6457.500
2,MB,2019-10-24,10500,0.77,6457.500,2,6457.500
3,MB,2019-10-25,10500,0.77,6457.500,3,6457.500
4,MB,2019-11-04,7875,0.77,4843.125,4,4843.125


In [54]:
min_intake_index = df_intake.sort_values(by='plot_rank').loc[(df_intake.unit_balance>0) & (df_intake.tat_start_date<=prod_end_date)].index[0]
input_unit = df_intake.loc[min_intake_index, 'unit_balance']
input_date = df_intake.loc[min_intake_index, 'tat_start_date']
print(min_intake_index, input_unit, input_date )

0 958.1700000000001 2019-10-16 00:00:00


In [55]:
min_output_index = df_output.sort_values(by='production_date').loc[(df_output.unit_balance>0) & (df_output.production_date>=input_date)].index[0]
output_unit = df_output.loc[min_output_index, 'unit_balance']
output_date = df_output.loc[min_output_index, 'production_date']
print(min_output_index, output_unit, output_date)

0 6457.5 2019-10-22 00:00:00


In [56]:
prod_plan = pd.DataFrame()

In [57]:
sc_days_diff = np.busday_count(prod_start_date.date(), prod_end_date.date(), weekmask=weekmask ,holidays=holidays)
sc_days_diff

9

In [58]:
df_intake['unit_balance'].sum()

48711.075

In [59]:
while df_intake['unit_balance'].sum()>0:
    #intake details collected
    min_intake_index = df_intake.sort_values(by='plot_rank').loc[(df_intake.unit_balance>0) & (df_intake.tat_start_date<=prod_end_date)].index
    if min_intake_index.size>0:
        min_intake_index = min_intake_index[0]
        input_unit = df_intake.loc[min_intake_index, 'unit_balance']
        input_date = df_intake.loc[min_intake_index, 'tat_start_date']
    else:
        print('All intake plotted!!')
        break
        
    #output details collected
    min_output_index = df_output.sort_values(by='production_date').loc[(df_output.unit_balance>0) & (df_output.production_date>=input_date)].index
    if min_output_index.size>0:
        min_output_index = min_output_index[0]
        output_unit = df_output.loc[min_output_index, 'unit_balance']
        output_date = df_output.loc[min_output_index, 'production_date']
    else:
        print('All planned output plotted!!. But there is still {} input units pending to plot!!' . format(df_intake['unit_balance'].sum()))
        break
    
    #check the so to sc_days to get min output numbers
    sc_days = np.busday_count(input_date.date(), output_date.date(), weekmask=weekmask ,holidays=holidays)
    
    #get min output based on sc_days
    min_output = df_min_output[(df_min_output.current_location==prod_plan_lab) & (df_min_output.so_to_sc_days==sc_days)]['stone_units'].values
    if min_output.size>0:
        min_output = min_output[0]
        #get the value to plot
        plot_unit = min(input_unit, output_unit, min_output)
        #update the unit balance details in intake and output table
        df_intake.loc[min_intake_index, 'unit_balance'] = df_intake.loc[min_intake_index, 'unit_balance'] - plot_unit
        df_output.loc[min_output_index, 'unit_balance'] = 0
    else:
        #if the min_output criteria is not available only take min between input and output
        plot_unit = min(input_unit, output_unit)
        #update the unit balance details in intake and output table
        df_intake.loc[min_intake_index, 'unit_balance'] = df_intake.loc[min_intake_index, 'unit_balance'] - plot_unit
        df_output.loc[min_output_index, 'unit_balance'] = df_output.loc[min_output_index, 'unit_balance'] - plot_unit
        
    #update prod plan table with new details
    prod_plan = prod_plan.append({'input_date': input_date, 'input_index': min_intake_index, 'output_date': output_date, 'output_index': min_output_index, 'plotted_unit': plot_unit}, ignore_index=True)
    
    
    #print('input_date: ', input_date , 'input_index: ', min_intake_index ,' output_date: ' , output_date , ' output_index: ', min_output_index, ' plotted_unit: ', plot_unit, ' input_unit: ', input_unit, ' output_unit: ', output_unit, ' min_output_unit: ', min_output, ' input_balance: ', df_intake.loc[min_intake_index, 'unit_balance'], ' output_balance ',  df_output.loc[min_output_index, 'unit_balance'])
    
    if df_intake['unit_balance'].sum()==0:
        print('All intake plotted!!')
        break
    else:
        continue

All planned output plotted!!. But there is still 1845.0 input units pending to plot!!


In [60]:
prod_plan['input_index'] = prod_plan['input_index'].astype(int)
prod_plan['output_index'] = prod_plan['output_index'].astype(int)
prod_plan['tat_target_date'] = prod_plan.input_index.map(df_intake['tat_target_date'].to_dict())
prod_plan['dd_mix'] = prod_plan.input_index.map(df_intake['dd_mix'].to_dict())
prod_plan['sc_to_sc_days'] = np.busday_count(pd.to_datetime(prod_plan['input_date']).values.astype('datetime64[D]'), pd.to_datetime(prod_plan['output_date']).values.astype('datetime64[D]'), weekmask=weekmask ,holidays=holidays)
prod_plan['stone_count'] = prod_plan.apply(lambda row: stone_unit_to_count(row) , axis=1)
prod_plan['tat_met'] = prod_plan.apply(lambda row: tat_met_calc(row) , axis=1)
prod_plan

,input_date,input_index,output_date,output_index,plotted_unit,tat_target_date,dd_mix,sc_to_sc_days,stone_count,tat_met
0,2019-10-16,0,2019-10-22,0,958.170,2019-10-18,0.77,3,1558.0,0.0
1,2019-10-17,1,2019-10-22,0,4883.100,2019-10-22,0.77,2,7940.0,7940.0
2,2019-10-18,2,2019-10-22,0,616.230,2019-10-23,0.77,1,1002.0,1002.0
3,2019-10-18,2,2019-10-23,1,4836.975,2019-10-23,0.77,2,7865.0,7865.0
4,2019-10-19,3,2019-10-23,1,1620.525,2019-10-23,0.77,1,2635.0,2635.0
5,2019-10-19,3,2019-10-24,2,1324.095,2019-10-23,0.77,2,2153.0,0.0
6,2019-10-22,4,2019-10-24,2,5133.405,2019-10-24,0.77,2,8347.0,8347.0
7,2019-10-22,4,2019-10-25,3,3151.875,2019-10-24,0.77,3,5125.0,0.0
8,2019-10-23,5,2019-10-25,3,3305.625,2019-10-25,0.77,2,5375.0,5375.0
9,2019-10-23,5,2019-11-04,4,1805.640,2019-10-25,0.77,3,2936.0,0.0


In [61]:
#starting inventory
p_start = df_intake[df_intake['tat_start_date']<prod_start_date]['stone_count'].sum()
#intake details grouped
p_in = df_intake.groupby(by='tat_start_date')['stone_count'].sum()
#output details grouped
p_out = prod_plan.groupby(by='output_date')['stone_count', 'tat_met'].sum()

In [62]:
prod_stats = pd.DataFrame(index=prod_date_range, data=p_out)
prod_stats['intake_stones'] = p_in
prod_stats.fillna(0, inplace=True)

In [63]:
prod_stats['cumilative_intake'] = prod_stats['intake_stones'].cumsum()
prod_stats['cumilative_output'] = prod_stats['stone_count'].cumsum()
prod_stats['ending_inventory'] = p_start + prod_stats['cumilative_intake'] - prod_stats['cumilative_output']
prod_stats['tat_met%'] = (prod_stats['tat_met'] / prod_stats['stone_count'])
prod_stats.fillna(0, inplace=True)

In [64]:
prod_stats

,stone_count,tat_met,intake_stones,cumilative_intake,cumilative_output,ending_inventory,tat_met%
2019-10-21,0.0,0.0,0.0,0.0,0.0,23153.0,0.000000
2019-10-22,10500.0,8942.0,13472.0,13472.0,10500.0,26125.0,0.851619
2019-10-23,10500.0,10500.0,8311.0,21783.0,21000.0,23936.0,1.000000
2019-10-24,10500.0,8347.0,6958.0,28741.0,31500.0,20394.0,0.794952
2019-10-25,10500.0,5375.0,12311.0,41052.0,42000.0,22205.0,0.511905
2019-10-26,0.0,0.0,0.0,41052.0,42000.0,22205.0,0.000000
2019-10-27,0.0,0.0,0.0,41052.0,42000.0,22205.0,0.000000
2019-10-28,0.0,0.0,0.0,41052.0,42000.0,22205.0,0.000000
2019-10-29,0.0,0.0,0.0,41052.0,42000.0,22205.0,0.000000
2019-10-30,0.0,0.0,0.0,41052.0,42000.0,22205.0,0.000000


In [65]:
index_alias = "Sr.No."
alias = ["tat_start_date", "output_date", "tat_target_date","sc_to_sc_days", "output_stone_count", "dd_mix", "tat_met_count"]
prod_plan_excel = prod_plan[['input_date', 'output_date', 'tat_target_date','sc_to_sc_days' ,'stone_count', 'dd_mix', 'tat_met']]
with pd.ExcelWriter('model_output.xlsx',engine='xlsxwriter', datetime_format='mm/dd/yyyy', date_format='mm/dd/yyyy') as writer:
    prod_plan_excel.to_excel(writer, sheet_name='Production_Plan', index=False, header=alias)
    prod_stats.to_excel(writer, sheet_name='Production_Plan_Performance', index_label='production_date')
    workbook = writer.book
    worksheet = writer.sheets["Production_Plan"]
    worksheet1 = writer.sheets["Production_Plan_Performance"]
    #set the column width as per your requirement
    worksheet.set_column('A:G', 18)
    worksheet1.set_column('A:G', 17)
    writer.save()